In [1]:
%pip install ucimlrepo
%pip install mlxtend
%pip install pandas
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
import matplotlib.pyplot as plt

In [3]:
# fetch dataset 
online_retail = fetch_ucirepo(id=352) 

In [4]:
# data (as pandas dataframes) 
X = online_retail.data.features 
y = online_retail.data.targets 
  
# metadata 
print(online_retail.metadata) 
  
# variable information 
print(online_retail.variables) 

{'uci_id': 352, 'name': 'Online Retail', 'repository_url': 'https://archive.ics.uci.edu/dataset/352/online+retail', 'data_url': 'https://archive.ics.uci.edu/static/public/352/data.csv', 'abstract': 'This is a transactional data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.', 'area': 'Business', 'tasks': ['Classification', 'Clustering'], 'characteristics': ['Multivariate', 'Sequential', 'Time-Series'], 'num_instances': 541909, 'num_features': 6, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': None, 'index_col': ['InvoiceNo', 'StockCode'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2015, 'last_updated': 'Mon Oct 21 2024', 'dataset_doi': '10.24432/C5BW33', 'creators': ['Daqing Chen'], 'intro_paper': {'ID': 361, 'type': 'NATIVE', 'title': 'Data mining for the online retail industry: A case study of RFM model-based customer segmenta

In [5]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
import matplotlib.pyplot as plt

# df = pd.DataFrame(X) # Importing data using the provided library
# df.head()

df = pd.read_csv('../data/Online_Retail.csv', encoding='unicode_escape')

In [6]:
df.dropna(subset=['Description', 'InvoiceNo'], inplace=True)
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]
df = df[df['Quantity'] > 0]  # Remove rows with negative quantities
df['InvoiceNo'] = df['InvoiceNo'].astype(str)

In [7]:
len(df.index)

530693

In [8]:
basket = (df.groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

def encode_units(x):
    return 1 if x >= 1 else 0

basket = basket.map(encode_units)

basket.head()

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,...,returned,taig adjust,test,to push order througha s stock was,website fixed,wrongly coded 20713,wrongly coded 23343,wrongly marked,wrongly marked 23343,wrongly sold (22719) barcode
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
min_supp = 0.01
min_thresh = 2

frequent_itemsets_fpgrowth = fpgrowth(basket, min_support=min_supp, use_colnames=True)
frequent_itemsets_fpgrowth = frequent_itemsets_fpgrowth.sort_values(by='support', ascending=False)

fpgrowth_rules = association_rules(frequent_itemsets_fpgrowth, metric='lift', min_threshold=min_thresh)

c:\Users\User\anaconda3\envs\stable\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


TypeError: association_rules() missing 1 required positional argument: 'num_itemsets'

In [ ]:
fpgrowth_rules['antecedents'] = fpgrowth_rules['antecedents'].apply(lambda x: ', '.join(list(x)))
fpgrowth_rules['consequents'] = fpgrowth_rules['consequents'].apply(lambda x: ', '.join(list(x)))

frequent_itemsets_fpgrowth.to_csv('../data/frequent_itemsets_fpgrowth.csv', index=False)
fpgrowth_rules.to_csv('../data/rules_fpgrowth.csv', index=False)

print(frequent_itemsets_fpgrowth.head())
print(fpgrowth_rules.head())

      support                              itemsets
0    0.112237  (WHITE HANGING HEART T-LIGHT HOLDER)
92   0.103894             (JUMBO BAG RED RETROSPOT)
297  0.098778            (REGENCY CAKESTAND 3 TIER)
587  0.083731                       (PARTY BUNTING)
42   0.077672             (LUNCH BAG RED RETROSPOT)
                        antecedents                       consequents  \
0           JUMBO BAG PINK POLKADOT           JUMBO BAG RED RETROSPOT   
1           JUMBO BAG RED RETROSPOT           JUMBO BAG PINK POLKADOT   
2   GREEN REGENCY TEACUP AND SAUCER  ROSES REGENCY TEACUP AND SAUCER    
3  ROSES REGENCY TEACUP AND SAUCER    GREEN REGENCY TEACUP AND SAUCER   
4            JUMBO STORAGE BAG SUKI           JUMBO BAG RED RETROSPOT   

   antecedent support  consequent support   support  confidence       lift  \
0            0.060489            0.103894  0.040971    0.677340   6.519558   
1            0.103894            0.060489  0.040971    0.394359   6.519558   
2            0.